In [1]:
import pandas as pd
import pylogit
import biogeme
from statsmodels.formula.api import logit
import numpy as np
import seaborn as sns
from tabulate import tabulate

In [2]:
sacog = pd.read_excel('sacog_recoded_for_qianhua.xlsx')

In [3]:
# Solve inconsistency in data coding and save as new dataset
sacog = sacog.replace('Never ', 'Never')
sacog.to_excel('sacog_recoded_012021.xlsx')

In [4]:
never = ["Never", "Almost never"]
never_and_noRes = ["Never", "Almost never", 'No Response']

def count_(cat):
    past_year_freq = 'PastYear-ECommerce-Frequency-' + cat
    may_freq = 'May-ECommerce-Frequency-' + cat
    may_order = 'May-ECommerce-Orders-' + cat
    oldUser = sacog[~sacog[past_year_freq].isin(never_and_noRes)]
    pastNever = sacog[sacog[past_year_freq].isin(never)]
    adopter = pastNever[(pastNever[may_order] > 0) | (~pastNever[may_freq].isin(never_and_noRes))]
    NonAdopter = pastNever[(pastNever[may_order] == 0) & pastNever[may_freq].isin(never)]
    quiter = oldUser[(oldUser[may_order] == 0) & (oldUser[may_freq].isin(never))]
    keeper = oldUser[(oldUser[may_order] > 0) | (~oldUser[may_freq].isin(never_and_noRes))]
    nowUser = sacog[(sacog[may_order] > 0) | (~sacog[may_freq].isin(never_and_noRes))]
    return [len(oldUser),
           len(pastNever), 
           len(adopter),
           len(NonAdopter),
           len(quiter),
           len(keeper),
           len(nowUser),
           oldUser,
           pastNever,
           adopter,
           NonAdopter,
           quiter,
           keeper,
           nowUser
           ]

dict_ = {}
cat_ = ['PreparedFood', 'Groceries', 'OtherFood', 'PaperCleaning', 'Clothing', 'HomeOffice', 'Medication', 'ChildcareItems', 'Other']
for cat in cat_:
    dict_[cat] = count_(cat)

In [5]:
def adoption(row, cat):
    may_freq = 'May-ECommerce-Frequency-' + cat
    may_order = 'May-ECommerce-Orders-' + cat
    if (row[may_order] > 0) | (row[may_freq] not in never_and_noRes):
        return 1 # Adopt/keeper
    elif (row[may_order] == 0) & (row[may_freq] in never):
        return 0 # Not Adopt/quiter
    else:
        return 999 # can't identify 

In [6]:
def oldnew(row, cat):
    past_year_freq = 'PastYear-ECommerce-Frequency-' + cat
    if (row[past_year_freq] not in never_and_noRes):
        return 1 # old
    elif (row[past_year_freq] in never):
        return 0 # new
    else:
        return 999 # can't identify 

In [7]:
freq_dict = {'More than 5 times per week': 7, 
             '3 - 5 times per week': 6,
             '2 - 3 times per week': 5,
             'About 1 time per week': 4,
             'Less than 1 time per week ': 3,
             'Less than 1 time per month ': 2,
             'Never': 1,
             'Almost never': 1,
             'No Response': 0}

In [8]:
def freq_online(orders):
    if orders > 5:
        return 7
    elif orders > 3:
        return 6
    elif orders >= 2:
        return 5
    elif orders >= 1:
        return 4

In [9]:
def freq_instore(trips):
    if trips/2 > 5:
        return 7
    elif trips/2 > 3:
        return 6
    elif trips/2 >= 2:
        return 5
    elif trips/2 >= 1:
        return 4
    elif trips/2 < 1:
        return 3

In [10]:
sacog['May_Freq_EPreparedFood'] = sacog['May-ECommerce-Frequency-PreparedFood'].map(freq_dict)
sacog['May_Freq_EGroceries'] = sacog['May-ECommerce-Frequency-Groceries'].map(freq_dict)
sacog['May_Freq_EOtherFood'] = sacog['May-ECommerce-Frequency-OtherFood'].map(freq_dict)
sacog['May_Freq_EPaperCleaning'] = sacog['May-ECommerce-Frequency-PaperCleaning'].map(freq_dict)
sacog['May_Freq_EClothing'] = sacog['May-ECommerce-Frequency-Clothing'].map(freq_dict)
sacog['May_Freq_EHomeOffice'] = sacog['May-ECommerce-Frequency-HomeOffice'].map(freq_dict)
sacog['May_Freq_EMedication'] = sacog['May-ECommerce-Frequency-Medication'].map(freq_dict)
sacog['May_Freq_EChildcareItems'] = sacog['May-ECommerce-Frequency-ChildcareItems'].map(freq_dict)
sacog['May_Freq_EOther'] = sacog['May-ECommerce-Frequency-Other'].map(freq_dict)

In [11]:
sacog['May_Freq_PreparedFood'] = sacog['May-InStore-Frequency-PreparedFood'].map(freq_dict)
sacog['May_Freq_Groceries'] = sacog['May-InStore-Frequency-Groceries'].map(freq_dict)
sacog['May_Freq_OtherFood'] = sacog['May-InStore-Frequency-OtherFood'].map(freq_dict)
sacog['May_Freq_PaperCleaning'] = sacog['May-InStore-Frequency-PaperCleaning'].map(freq_dict)
sacog['May_Freq_Clothing'] = sacog['May-InStore-Frequency-Clothing'].map(freq_dict)
sacog['May_Freq_HomeOffice'] = sacog['May-InStore-Frequency-HomeOffice'].map(freq_dict)
sacog['May_Freq_Medication'] = sacog['May-InStore-Frequency-Medication'].map(freq_dict)
sacog['May_Freq_ChildcareItems'] = sacog['May-InStore-Frequency-ChildcareItems'].map(freq_dict)
sacog['May_Freq_Other'] = sacog['May-InStore-Frequency-Other'].map(freq_dict)

In [12]:
# Impute from # of orders during survey week
sacog.loc[sacog['May-ECommerce-Orders-PreparedFood'] > 0, 'May_Freq_EPreparedFood'] = sacog.loc[sacog['May-ECommerce-Orders-PreparedFood'] > 0]['May-ECommerce-Orders-PreparedFood'].apply(freq_online)
sacog.loc[sacog['May-ECommerce-Orders-Groceries'] > 0, 'May_Freq_EGroceries'] = sacog.loc[sacog['May-ECommerce-Orders-Groceries'] > 0]['May-ECommerce-Orders-Groceries'].apply(freq_online)
sacog.loc[sacog['May-ECommerce-Orders-OtherFood'] > 0, 'May_Freq_EOtherFood'] = sacog.loc[sacog['May-ECommerce-Orders-OtherFood'] > 0]['May-ECommerce-Orders-OtherFood'].apply(freq_online)
sacog.loc[sacog['May-ECommerce-Orders-PaperCleaning'] > 0, 'May_Freq_EPaperCleaning'] = sacog.loc[sacog['May-ECommerce-Orders-PaperCleaning'] > 0]['May-ECommerce-Orders-PaperCleaning'].apply(freq_online)
sacog.loc[sacog['May-ECommerce-Orders-Clothing'] > 0, 'May_Freq_EClothing'] = sacog.loc[sacog['May-ECommerce-Orders-Clothing'] > 0]['May-ECommerce-Orders-Clothing'].apply(freq_online)
sacog.loc[sacog['May-ECommerce-Orders-HomeOffice'] > 0, 'May_Freq_EHomeOffice'] = sacog.loc[sacog['May-ECommerce-Orders-HomeOffice'] > 0]['May-ECommerce-Orders-HomeOffice'].apply(freq_online)
sacog.loc[sacog['May-ECommerce-Orders-Medication'] > 0, 'May_Freq_EMedication'] = sacog.loc[sacog['May-ECommerce-Orders-Medication'] > 0]['May-ECommerce-Orders-Medication'].apply(freq_online)
sacog.loc[sacog['May-ECommerce-Orders-ChildcareItems'] > 0, 'May_Freq_EChildcareItems'] = sacog.loc[sacog['May-ECommerce-Orders-ChildcareItems'] > 0]['May-ECommerce-Orders-ChildcareItems'].apply(freq_online)
sacog.loc[sacog['May-ECommerce-Orders-Other'] > 0, 'May_Freq_EOther'] = sacog.loc[sacog['May-ECommerce-Orders-Other'] > 0]['May-ECommerce-Orders-Other'].apply(freq_online)

In [13]:
# Impute from # of trips during survey week
sacog.loc[sacog['May-InStore-Trips-PreparedFood'] > 0, 'May_Freq_PreparedFood'] = sacog.loc[sacog['May-InStore-Trips-PreparedFood'] > 0]['May-InStore-Trips-PreparedFood'].apply(freq_instore)
sacog.loc[sacog['May-InStore-Trips-Groceries'] > 0, 'May_Freq_Groceries'] = sacog.loc[sacog['May-InStore-Trips-Groceries'] > 0]['May-InStore-Trips-Groceries'].apply(freq_instore)
sacog.loc[sacog['May-InStore-Trips-OtherFood'] > 0, 'May_Freq_OtherFood'] = sacog.loc[sacog['May-InStore-Trips-OtherFood'] > 0]['May-InStore-Trips-OtherFood'].apply(freq_instore)
sacog.loc[sacog['May-InStore-Trips-PaperCleaning'] > 0, 'May_Freq_PaperCleaning'] = sacog.loc[sacog['May-InStore-Trips-PaperCleaning'] > 0]['May-InStore-Trips-PaperCleaning'].apply(freq_instore)
sacog.loc[sacog['May-InStore-Trips-Clothing'] > 0, 'May_Freq_Clothing'] = sacog.loc[sacog['May-InStore-Trips-Clothing'] > 0]['May-InStore-Trips-Clothing'].apply(freq_instore)
sacog.loc[sacog['May-InStore-Trips-HomeOffice'] > 0, 'May_Freq_HomeOffice'] = sacog.loc[sacog['May-InStore-Trips-HomeOffice'] > 0]['May-InStore-Trips-HomeOffice'].apply(freq_instore)
sacog.loc[sacog['May-InStore-Trips-Medication'] > 0, 'May_Freq_Medication'] = sacog.loc[sacog['May-InStore-Trips-Medication'] > 0]['May-InStore-Trips-Medication'].apply(freq_instore)
sacog.loc[sacog['May-InStore-Trips-ChildcareItems'] > 0, 'May_Freq_ChildcareItems'] = sacog.loc[sacog['May-InStore-Trips-ChildcareItems'] > 0]['May-InStore-Trips-ChildcareItems'].apply(freq_instore)
sacog.loc[sacog['May-InStore-Trips-Other'] > 0, 'May_Freq_Other'] = sacog.loc[sacog['May-InStore-Trips-Other'] > 0]['May-InStore-Trips-Other'].apply(freq_instore)

#### Change sharing indicator encode to reflect actual responsibility sharing status.

In [14]:
sacog.loc[sacog['May-ShareShopping'].isin(['Yes, for all shopping needs']), 'May-Shared-PreparedFood'] = 1
sacog.loc[sacog['May-ShareShopping'].isin(['Yes, for all shopping needs']), 'May-Shared-Groceries'] = 1
sacog.loc[sacog['May-ShareShopping'].isin(['Yes, for all shopping needs']), 'May-Shared-OtherFood'] = 1
sacog.loc[sacog['May-ShareShopping'].isin(['Yes, for all shopping needs']), 'May-Shared-PaperCleaning'] = 1
sacog.loc[sacog['May-ShareShopping'].isin(['Yes, for all shopping needs']), 'May-Shared-Clothing'] = 1
sacog.loc[sacog['May-ShareShopping'].isin(['Yes, for all shopping needs']), 'May-Shared-HomeOffice'] = 1
sacog.loc[sacog['May-ShareShopping'].isin(['Yes, for all shopping needs']), 'May-Shared-Medication'] = 1
sacog.loc[sacog['May-ShareShopping'].isin(['Yes, for all shopping needs']), 'May-Shared-ChildcareItems'] = 1
sacog.loc[sacog['May-ShareShopping'].isin(['Yes, for all shopping needs']), 'May-Shared-Other'] = 1

In [15]:
sacog.to_excel('sacog_recode_freq_032421.xlsx')

## 1) E-prepared-food

In [34]:
pastNeverF = count_('PreparedFood')[8]
oldUserF = count_('PreparedFood')[7]
nowUserF = count_('PreparedFood')[13]

In [35]:
pastNeverF['adopt'] = pastNeverF.apply(lambda row: adoption(row, 'PreparedFood'), axis = 1)
oldUserF['keep'] = oldUserF.apply(lambda row: adoption(row, 'PreparedFood'), axis = 1)
nowUserF['old'] = nowUserF.apply(lambda row: oldnew(row, 'PreparedFood'), axis = 1)

/Users/apple/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/Users/apple/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/apple/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the docum

In [24]:
list(sacog.columns)

['Unnamed: 0',
 'Duration (in seconds)',
 'Response ID',
 'May-WorkingHours',
 'May-TelecommutingDays',
 'May-Trips-Commuting',
 'May-Trips-Medical',
 'May-Trips-Gym',
 'May-Trips-ToSchool',
 'May-Trips-FromSchool',
 'May-Trips-Bar',
 'May-Trips-MovieTheater',
 'May-Trips-Friends',
 'May-Trips-Family',
 'May-Trips-Other',
 'May-Trips-OtherType',
 'CommutingFollowup',
 'May-TripDistance-Commuting',
 'May-TripDistance-Medical',
 'May-TripDistance-Gym',
 'May-TripDistance-ToSchool',
 'May-TripDistance-FromSchool',
 'May-TripDistance-Bar',
 'May-TripDistance-MovieTheater',
 'May-TripDistance-Friends',
 'May-TripDistance-Family',
 'May-TripDistance-Other',
 'May-TripDistance-OtherType',
 'May-TripMode-Commuting',
 'May-TripMode-Medical',
 'May-TripMode-Gym',
 'May-TripMode-ToSchool',
 'May-TripMode-FromSchool',
 'May-TripMode-Bar',
 'May-TripMode-MovieTheater',
 'May-TripMode-Friends',
 'May-TripMode-Family',
 'May-TripMode-Other',
 'May-TripMode-OtherType',
 'LivingInSamePlace',
 'JanFeb-W

In [25]:
ls1 = ['Old User',
                 nowUserF[(nowUserF['old'] == 1) & (~nowUserF['May-InStore-PreparedFood'].isin([-9998]))]['May-InStore-PreparedFood'].mean(), 
                 nowUserF[(nowUserF['old'] == 1) & (~nowUserF['May-Curbside-PreparedFood'].isin([-9998]))]['May-Curbside-PreparedFood'].mean(), 
                 nowUserF[(nowUserF['old'] == 1) & (~nowUserF['May-ToHome-PreparedFood'].isin([-9998]))]['May-ToHome-PreparedFood'].mean(), 
                 nowUserF[(nowUserF['old'] == 1) & (~nowUserF['May-ToNonHome-PreparedFood'].isin([-9998]))]['May-ToNonHome-PreparedFood'].mean(),
                 nowUserF[(nowUserF['old'] == 1) & (~nowUserF['May-ECommerce-Orders-PreparedFood'].isin([-9998]))]['May-ECommerce-Orders-PreparedFood'].mean(),
       nowUserF[(nowUserF['old'] == 1) & (~nowUserF['May-InStore-Trips-PreparedFood'].isin([-9998]))]['May-InStore-Trips-PreparedFood'].mean()
                ]
ls2 = ['New User',
                 nowUserF[(nowUserF['old'] == 0) & (~nowUserF['May-InStore-PreparedFood'].isin([-9998]))]['May-InStore-PreparedFood'].mean(), 
                 nowUserF[(nowUserF['old'] == 0) & (~nowUserF['May-Curbside-PreparedFood'].isin([-9998]))]['May-Curbside-PreparedFood'].mean(), 
                 nowUserF[(nowUserF['old'] == 0) & (~nowUserF['May-ToHome-PreparedFood'].isin([-9998]))]['May-ToHome-PreparedFood'].mean(), 
                 nowUserF[(nowUserF['old'] == 0) & (~nowUserF['May-ToNonHome-PreparedFood'].isin([-9998]))]['May-ToNonHome-PreparedFood'].mean(),
                 nowUserF[(nowUserF['old'] == 0) & (~nowUserF['May-ECommerce-Orders-PreparedFood'].isin([-9998]))]['May-ECommerce-Orders-PreparedFood'].mean(),
       nowUserF[(nowUserF['old'] == 0) & (~nowUserF['May-InStore-Trips-PreparedFood'].isin([-9998]))]['May-InStore-Trips-PreparedFood'].mean()
                ]
ls3 = ['Non adopter', '-','-','-','-','-',
      pastNeverF[(pastNeverF['adopt'] == 0) & (~pastNeverF['May-InStore-Trips-PreparedFood'].isin([-9998]))]['May-InStore-Trips-PreparedFood'].mean()]

#### Old users of E-preparedfood have more orders compared to new users during the surveyed week. And their trips to purchase prepared food instore is also higher than new user, and even non-adopter. For prepared food, non adopter originally are not that into restuarant food, therefore, their also make fewer in store trips compared to adopter. Old users of E-preparedfood are fan of restuarant food. They both do more in store purchase and online order.

In [26]:
print(tabulate([[ls1[0]] + list(np.round(np.array(ls1[1:]), 2)), 
                [ls2[0]] + list(np.round(np.array(ls2[1:]), 2)), 
                ls3[:6] + [np.round(ls3[6], 2)]], 
               headers=['User Type', 'InStore pickup', 'Curbside pickup', 'To home delivery', 'To nonhome delivery', 'EOrders', 'InStore Trips']))

User Type    InStore pickup    Curbside pickup    To home delivery    To nonhome delivery    EOrders      InStore Trips
-----------  ----------------  -----------------  ------------------  ---------------------  ---------  ---------------
Old User     1.19              0.83               0.84                0.08                   1.58                  2.26
New User     0.72              0.75               0.44                0.0                    1.29                  1.96
Non adopter  -                 -                  -                   -                      -                     1.49


### 1.2 Binary choice model for initial adoption

#### Both socio-demo and attitude

In [27]:
sacog[sacog['May-InStore-Mode-PreparedFood'] != -9998]['May-InStore-Mode-PreparedFood'].value_counts() # change to one hot encoding
sacog[sacog['May-InStore-Distance-PreparedFood'] != -9998]['May-InStore-Distance-PreparedFood'].value_counts() # ordered categorical or one hot indicator?
sacog[~sacog['Change-Shopping-TripsToStore-PreparedFood'].isin(['No Response'])]['Change-Shopping-TripsToStore-PreparedFood'].value_counts()
sacog[~sacog['Change-Shopping-InStorePurchaseSize-PreparedFood'].isin(['No Response'])]['Change-Shopping-InStorePurchaseSize-PreparedFood'].value_counts()
sacog[~sacog['Change-Shopping-TripDistanceToStore-PreparedFood'].isin(['No Response'])]['Change-Shopping-TripDistanceToStore-PreparedFood'].value_counts()


pastNeverF[(pastNeverF['adopt'] == 1) & (~pastNeverF['Change-Shopping-TripsToStore-PreparedFood'].isin(['No Response']))]['Change-Shopping-TripsToStore-PreparedFood'].value_counts(),\
pastNeverF[(pastNeverF['adopt'] == 0) & (~pastNeverF['Change-Shopping-TripsToStore-PreparedFood'].isin(['No Response']))]['Change-Shopping-TripsToStore-PreparedFood'].value_counts()

(Less         21
 No change    17
 More         11
 Name: Change-Shopping-TripsToStore-PreparedFood, dtype: int64, No change    26
 Less         14
 More          3
 Name: Change-Shopping-TripsToStore-PreparedFood, dtype: int64)

In [36]:
pastNeverF = pd.concat([pastNeverF, pd.get_dummies(pastNeverF['Change-Shopping-TripsToStore-PreparedFood'], prefix='InstoreTrips')], axis = 1)
pastNeverF = pd.concat([pastNeverF, pd.get_dummies(pastNeverF['Change-Shopping-InStorePurchaseSize-PreparedFood'], prefix='InstoreSize')], axis = 1)
pastNeverF = pd.concat([pastNeverF, pd.get_dummies(pastNeverF['Change-Shopping-TripDistanceToStore-PreparedFood'], prefix='InstoreDist')], axis = 1)
pastNeverF = pd.concat([pastNeverF, pd.get_dummies(pastNeverF['May-InStore-Mode-PreparedFood'], prefix='InstoreMode')], axis = 1)

In [40]:
list(pastNeverF.columns)

['Unnamed: 0',
 'Duration (in seconds)',
 'Response ID',
 'May-WorkingHours',
 'May-TelecommutingDays',
 'May-Trips-Commuting',
 'May-Trips-Medical',
 'May-Trips-Gym',
 'May-Trips-ToSchool',
 'May-Trips-FromSchool',
 'May-Trips-Bar',
 'May-Trips-MovieTheater',
 'May-Trips-Friends',
 'May-Trips-Family',
 'May-Trips-Other',
 'May-Trips-OtherType',
 'CommutingFollowup',
 'May-TripDistance-Commuting',
 'May-TripDistance-Medical',
 'May-TripDistance-Gym',
 'May-TripDistance-ToSchool',
 'May-TripDistance-FromSchool',
 'May-TripDistance-Bar',
 'May-TripDistance-MovieTheater',
 'May-TripDistance-Friends',
 'May-TripDistance-Family',
 'May-TripDistance-Other',
 'May-TripDistance-OtherType',
 'May-TripMode-Commuting',
 'May-TripMode-Medical',
 'May-TripMode-Gym',
 'May-TripMode-ToSchool',
 'May-TripMode-FromSchool',
 'May-TripMode-Bar',
 'May-TripMode-MovieTheater',
 'May-TripMode-Friends',
 'May-TripMode-Family',
 'May-TripMode-Other',
 'May-TripMode-OtherType',
 'LivingInSamePlace',
 'JanFeb-W

In [149]:
adoptF = pastNeverF[pastNeverF['adopt'] != 999][['adopt',
                                                 'May-TelecommutingDays',
                                                 'NumberInHousehold',
                                                 'LiveWithSpouse',
                                                 'LiveWithChildren',
                                                 #'May-Shared-PreparedFood',
                                                 'May_Freq_PreparedFood',
                                                 'May-Ease-PreparedFood',
                                                 #'InstoreTrips_Less',
                                                 #'InstoreSize_Less',
                                                 #'InstoreDist_Less',
                                                 #'InstoreMode_Drive alone',
                                                'COVID-PPEReducesContraction', 'COVID-PPEReducesTransmission', 
                  'COVID-AbleSocialDistance', 'Trust-FoodDeliveryCompanies',
                  'Trust-FoodDeliveryDrivers', 
                                                 #'Behavior-PickupExcuseToGetOut',
                  'Behavior-NotMindRestaurantOnline', 
                  'ECommerce-Food-HardToKnowExactly', 'ECommerce-Food-WorryAboutTheft', 
                                                 #'ECommerce-Food-SavesTime', 
                                                 'Online-Ease']] 

In [150]:
# Remove record with response -9998 or 997, reduce to 89 records
adoptF = adoptF[(~np.isin(adoptF, [-9998, 997,-1]).any(axis=1)) & (adoptF.May_Freq_PreparedFood != 0)]

In [151]:
import statsmodels.api as sm

X = adoptF.iloc[:,1:]
Y = adoptF.iloc[:,0]
X = sm.add_constant(X) #?
est = sm.Logit(Y, X).fit()
print(est.summary())


Optimization terminated successfully.
         Current function value: 0.306086
         Iterations 9
                           Logit Regression Results                           
Dep. Variable:                  adopt   No. Observations:                   42
Model:                          Logit   Df Residuals:                       26
Method:                           MLE   Df Model:                           15
Date:                Sun, 28 Mar 2021   Pseudo R-squ.:                  0.4677
Time:                        17:11:03   Log-Likelihood:                -12.856
converged:                       True   LL-Null:                       -24.152
Covariance Type:            nonrobust   LLR p-value:                   0.09319
                                       coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------------------
const                                1.1264     10.257      0.11

/Users/apple/anaconda3/lib/python3.6/site-packages/numpy/core/fromnumeric.py:2580: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


#### Only socio-demo

In [152]:
adoptF = pastNeverF[pastNeverF['adopt'] != 999][['adopt',
                                                 'May-TelecommutingDays',
                                                 'NumberInHousehold',
                                                 'LiveWithSpouse',
                                                 'LiveWithChildren',
                                                 #'May-Shared-PreparedFood',
                                                 'May_Freq_PreparedFood',
                                                 'May-Ease-PreparedFood',
                                                'InstoreTrips_Less',
                                                 'InstoreSize_Less',
                                                 'InstoreDist_Less',
                                                 'InstoreMode_Drive alone'
                                                 ]] 

In [153]:
# Remove record with response -9998 or 997, reduce to 89 records
adoptF = adoptF[(~np.isin(adoptF, [-9998, 997,-1]).any(axis=1)) & (adoptF.May_Freq_PreparedFood != 0)]

#### Household with spouse, children are more likely to adopt; household with shared responsibility of shopping are less likely to adopt; with a higher frequency of in-store prepared food shopping are less likely to adopt. Household that share shopping burden are less likely to adopt. More people in the household, less likely to adopt.

In [154]:
X = adoptF.iloc[:,1:]
Y = adoptF.iloc[:,0]
X = sm.add_constant(X) #?
est = sm.Logit(Y, X).fit()
print(est.summary())

         Current function value: 0.320168
         Iterations: 35
                           Logit Regression Results                           
Dep. Variable:                  adopt   No. Observations:                   42
Model:                          Logit   Df Residuals:                       31
Method:                           MLE   Df Model:                           10
Date:                Sun, 28 Mar 2021   Pseudo R-squ.:                  0.4432
Time:                        17:12:45   Log-Likelihood:                -13.447
converged:                      False   LL-Null:                       -24.152
Covariance Type:            nonrobust   LLR p-value:                   0.01841
                              coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------------
const                       1.9687      3.718      0.530      0.596      -5.318       9.256
May-TelecommutingDays     

/Users/apple/anaconda3/lib/python3.6/site-packages/numpy/core/fromnumeric.py:2580: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
/Users/apple/anaconda3/lib/python3.6/site-packages/statsmodels/base/model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)


#### Only attitude

In [155]:
adoptF = pastNeverF[pastNeverF['adopt'] != 999][['adopt', 'COVID-PPEReducesContraction', 'COVID-PPEReducesTransmission', 
                  'COVID-AbleSocialDistance', 'Trust-FoodDeliveryCompanies',
                  'Trust-FoodDeliveryDrivers', 'Behavior-PickupExcuseToGetOut',
                  'Behavior-NotMindRestaurantOnline', 'ECommerce-Food-HardToKnowExactly',
                  'ECommerce-Food-WorryAboutTheft', 'Online-Ease']]

In [156]:
adoptF = adoptF[(~np.isin(adoptF, [-9998, 997,-1]).any(axis=1))]

#### People who see pick up as an excuse to get out, more likely to adopt; not mind restaurant online, more likely to adopt; worry about theft, less likely to adopt. Online ease (?). I find on average, adopter has a more positive view towards online ease than non-adopter, but the sign is reversed. Why?

In [157]:
X = adoptF.iloc[:,1:]
Y = adoptF.iloc[:,0]
X = sm.add_constant(X) #?
est = sm.Logit(Y, X).fit()
print(est.summary())

Optimization terminated successfully.
         Current function value: 0.512995
         Iterations 6
                           Logit Regression Results                           
Dep. Variable:                  adopt   No. Observations:                   89
Model:                          Logit   Df Residuals:                       78
Method:                           MLE   Df Model:                           10
Date:                Sun, 28 Mar 2021   Pseudo R-squ.:                  0.2544
Time:                        17:16:50   Log-Likelihood:                -45.657
converged:                       True   LL-Null:                       -61.234
Covariance Type:            nonrobust   LLR p-value:                 0.0005530
                                       coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------------------
const                               -2.8752      2.953     -0.97

/Users/apple/anaconda3/lib/python3.6/site-packages/numpy/core/fromnumeric.py:2580: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


### 1.3 Binary choice model for exit behavior

In [88]:
keepF = oldUserF[oldUserF['keep'] != 999][['keep',
                                                 'May-TelecommutingDays',
                                                 'NumberInHousehold',
                                                 'LiveWithSpouse',
                                                 'LiveWithChildren',
                                                 'LiveWithParents',
                                                 'May-Shared-PreparedFood',
                                                 'May_Freq_PreparedFood',
                                                 'May-Ease-PreparedFood',
                                          'COVID-PPEReducesContraction', 'COVID-PPEReducesTransmission', 
                  'COVID-AbleSocialDistance', 'Trust-FoodDeliveryCompanies',
                  'Trust-FoodDeliveryDrivers', 'Behavior-PickupExcuseToGetOut',
                  'Behavior-NotMindRestaurantOnline', 'ECommerce-Food-HardToKnowExactly',
                  'ECommerce-Food-WorryAboutTheft', 'Attitude-HaveEnoughFood', 'Online-Ease']]

In [89]:
keepF = keepF[~np.isin(keepF, [-9998, 997,-1]).any(axis=1)]

In [90]:
keepF.describe()

,keep,May-TelecommutingDays,NumberInHousehold,LiveWithSpouse,LiveWithChildren,LiveWithParents,May-Shared-PreparedFood,May_Freq_PreparedFood,May-Ease-PreparedFood,COVID-PPEReducesContraction,COVID-PPEReducesTransmission,COVID-AbleSocialDistance,Trust-FoodDeliveryCompanies,Trust-FoodDeliveryDrivers,Behavior-PickupExcuseToGetOut,Behavior-NotMindRestaurantOnline,ECommerce-Food-HardToKnowExactly,ECommerce-Food-WorryAboutTheft,Attitude-HaveEnoughFood,Online-Ease
count,94.000000,94.000000,94.000000,94.000000,94.000000,94.000000,94.000000,94.000000,94.000000,94.000000,94.000000,94.000000,94.000000,94.000000,94.000000,94.000000,94.000000,94.000000,94.000000,94.000000
mean,0.957447,3.500000,2.425532,0.659574,0.351064,0.031915,0.500000,3.914894,4.574468,4.106383,4.531915,4.234043,3.265957,3.787234,2.861702,4.106383,3.765957,2.734043,4.574468,4.202128
std,0.202930,2.198484,1.159249,0.476393,0.479862,0.176716,0.502681,1.300617,0.754761,1.031421,0.728854,0.709771,0.929511,0.685837,1.187761,0.782443,0.872834,1.099125,0.518255,0.770139
min,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000,1.000000,2.000000,2.000000,1.000000,2.000000,1.000000,2.000000,2.000000,1.000000,3.000000,2.000000
25%,1.000000,1.250000,2.000000,0.000000,0.000000,0.000000,0.000000,4.000000,4.000000,4.000000,4.000000,4.000000,3.000000,3.000000,2.000000,4.000000,3.000000,2.000000,4.000000,4.000000
50%,1.000000,5.000000,2.000000,1.000000,0.000000,0.000000,0.500000,4.000000,5.000000,4.000000,5.000000,4.000000,3.000000,4.000000,3.000000,4.000000,4.000000,2.000000,5.000000,4.000000
75%,1.000000,5.000000,3.000000,1.000000,1.000000,0.000000,1.000000,5.000000,5.000000,5.000000,5.000000,5.000000,4.000000,4.000000,4.000000,5.000000,4.000000,4.000000,5.000000,5.000000
max,1.000000,6.000000,5.000000,1.000000,1.000000,1.000000,1.000000,7.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000


In [91]:
X = keepF.iloc[:,1:]
Y = keepF.iloc[:,0]
X = sm.add_constant(X) #?
est = sm.Logit(Y, X).fit()
print(est.summary())

         Current function value: 0.085835
         Iterations: 35
                           Logit Regression Results                           
Dep. Variable:                   keep   No. Observations:                   94
Model:                          Logit   Df Residuals:                       74
Method:                           MLE   Df Model:                           19
Date:                Sun, 28 Mar 2021   Pseudo R-squ.:                  0.5122
Time:                        16:05:08   Log-Likelihood:                -8.0684
converged:                      False   LL-Null:                       -16.542
Covariance Type:            nonrobust   LLR p-value:                    0.5935
                                       coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------------------
const                              -94.1731   3926.895     -0.024      0.981   -7790.745    7602.399

/Users/apple/anaconda3/lib/python3.6/site-packages/numpy/core/fromnumeric.py:2580: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
/Users/apple/anaconda3/lib/python3.6/site-packages/statsmodels/base/model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)


## 2) E-grocery

In [ ]:
'JanFeb-ShareGreoceries',
 'JanFeb-GroceriesFor-Myself',
 'JanFeb-GroceriesFor-ImmediateFamily',
 'JanFeb-GroceriesFor-ExtendedFamily',
 'JanFeb-GroceriesFor-NonFamilyInHousehold',
 'JanFeb-GroceriesFor-NonFamilyNotInHousehold',
 'JanFeb-GroceriesFor-Colleagues',
 'JanFeb-GroceriesFor-Other',
 'JanFeb-GroceriesFor-OtherType',
 'JanFeb-Grocery-Frequency',
 'JanFeb-Grocery-MultipleStores',
 'JanFeb-Grocery-OtherErrands',
 'JanFeb-Grocery-ToFromWork',
 'JanFeb-Grocery-DayOfWeek',
#Binary choice model for adoption behavior past year.

In [92]:
pastNeverG = count_('Groceries')[8]
oldUserG = count_('Groceries')[7]
nowUserG = count_('Groceries')[13]

In [93]:
pastNeverG['adopt'] = pastNeverG.apply(lambda row: adoption(row, 'Groceries'), axis = 1)
oldUserG['keep'] = oldUserG.apply(lambda row: adoption(row, 'Groceries'), axis = 1)
nowUserG['old'] = nowUserG.apply(lambda row: oldnew(row, 'Groceries'), axis = 1)

/Users/apple/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/Users/apple/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/apple/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the docum

In [94]:
ls1 = ['Old User',
                 nowUserG[(nowUserG['old'] == 1) & (~nowUserG['May-InStore-Groceries'].isin([-9998]))]['May-InStore-Groceries'].mean(), 
                 nowUserG[(nowUserG['old'] == 1) & (~nowUserG['May-Curbside-Groceries'].isin([-9998]))]['May-Curbside-Groceries'].mean(), 
                 nowUserG[(nowUserG['old'] == 1) & (~nowUserG['May-ToHome-Groceries'].isin([-9998]))]['May-ToHome-Groceries'].mean(), 
                 nowUserG[(nowUserG['old'] == 1) & (~nowUserG['May-ToNonHome-Groceries'].isin([-9998]))]['May-ToNonHome-Groceries'].mean(),
                 nowUserG[(nowUserG['old'] == 1) & (~nowUserG['May-ECommerce-Orders-Groceries'].isin([-9998]))]['May-ECommerce-Orders-Groceries'].mean(),
       nowUserG[(nowUserG['old'] == 1) & (~nowUserG['May-InStore-Trips-Groceries'].isin([-9998]))]['May-InStore-Trips-Groceries'].mean()/2
                ]
ls2 = ['New User',
                 nowUserG[(nowUserG['old'] == 0) & (~nowUserG['May-InStore-Groceries'].isin([-9998]))]['May-InStore-Groceries'].mean(), 
                 nowUserG[(nowUserG['old'] == 0) & (~nowUserG['May-Curbside-Groceries'].isin([-9998]))]['May-Curbside-Groceries'].mean(), 
                 nowUserG[(nowUserG['old'] == 0) & (~nowUserG['May-ToHome-Groceries'].isin([-9998]))]['May-ToHome-Groceries'].mean(), 
                 nowUserG[(nowUserG['old'] == 0) & (~nowUserG['May-ToNonHome-Groceries'].isin([-9998]))]['May-ToNonHome-Groceries'].mean(),
                 nowUserG[(nowUserG['old'] == 0) & (~nowUserG['May-ECommerce-Orders-Groceries'].isin([-9998]))]['May-ECommerce-Orders-Groceries'].mean(),
       nowUserG[(nowUserG['old'] == 0) & (~nowUserG['May-InStore-Trips-Groceries'].isin([-9998]))]['May-InStore-Trips-Groceries'].mean()/2
                ]
ls3 = ['Non adopter', '-','-','-','-','-',
      pastNeverG[(pastNeverG['adopt'] == 0) & (~pastNeverG['May-InStore-Trips-Groceries'].isin([-9998]))]['May-InStore-Trips-Groceries'].mean()/2]

In [95]:
[ls1[0]] + list(np.round(np.array(ls1[1:]), 2))

['Old User', 0.28, 0.38, 1.03, 0.0, 1.07, 1.41]

### For grocery, online order is a substitute for in store trips. Old Users satisfies a larger portion of their demand for groceries using online orders while for new users, in store purchase still takes up a larger portion. Non adopters make the most in store trips.

In [127]:
print(tabulate([[ls1[0]] + list(np.round(np.array(ls1[1:]), 2)), 
                [ls2[0]] + list(np.round(np.array(ls2[1:]), 2)), 
                ls3[:6] + [np.round(ls3[6], 2)]], 
               headers=['User Type', 'InStore pickup', 'Curbside pickup', 'To home delivery', 'To nonhome delivery', 'EOrders', 'InStore Trips']))

User Type    InStore pickup    Curbside pickup    To home delivery    To nonhome delivery    EOrders      InStore Trips
-----------  ----------------  -----------------  ------------------  ---------------------  ---------  ---------------
Old User     0.28              0.38               1.03                0.0                    1.07                  1.41
New User     0.21              0.68               0.92                0.12                   0.98                  1.37
Non adopter  -                 -                  -                   -                      -                     1.55


In [100]:
pastNeverG = pd.concat([pastNeverG, pd.get_dummies(pastNeverG['Change-Shopping-TripsToStore-Groceries'], prefix='InstoreTrips')], axis = 1)
pastNeverG = pd.concat([pastNeverG, pd.get_dummies(pastNeverG['Change-Shopping-InStorePurchaseSize-Groceries'], prefix='InstoreSize')], axis = 1)
pastNeverG = pd.concat([pastNeverG, pd.get_dummies(pastNeverG['Change-Shopping-TripDistanceToStore-Groceries'], prefix='InstoreDist')], axis = 1)
pastNeverG = pd.concat([pastNeverG, pd.get_dummies(pastNeverG['May-InStore-Mode-Groceries'], prefix='InstoreMode')], axis = 1)

In [158]:
adoptG = pastNeverG[pastNeverG['adopt'] != 999][['adopt',
                                                 'May-TelecommutingDays',
                                                 'NumberInHousehold',
                                                 'LiveWithSpouse',
                                                 'LiveWithChildren',
                                                 'LiveWithParents',
                                                 #'LiveWithGrandparents',
                                                 'May-Shared-Groceries',
                                                 'May_Freq_Groceries',
                                                 'May-Ease-Groceries',
                                                 'InstoreTrips_Less',
                                                 'InstoreSize_Less',
                                                 'InstoreDist_Less',
                                                 'InstoreMode_Drive alone',
                                                'COVID-PPEReducesContraction', 'COVID-PPEReducesTransmission', 
                  'COVID-AbleSocialDistance', 'Trust-GroceryDeliveryCompanies',
                  'Trust-GroceryDeliveryDrivers', 'Behavior-GroceryExcuseToGetOut',
                  'Behavior-NotMindGroceriesOnline', 'Behavior-InPersonIsChore', 
                  'ECommerce-Food-HardToKnowExactly', 'ECommerce-Food-WorryAboutTheft', 
                 'Online-Ease']]

In [159]:
adoptG = adoptG[~np.isin(adoptG, [-9998, 997,-1]).any(axis=1)]

In [160]:
adoptG.describe()

,adopt,May-TelecommutingDays,NumberInHousehold,LiveWithSpouse,LiveWithChildren,LiveWithParents,May-Shared-Groceries,May_Freq_Groceries,May-Ease-Groceries,InstoreTrips_Less,...,COVID-PPEReducesTransmission,COVID-AbleSocialDistance,Trust-GroceryDeliveryCompanies,Trust-GroceryDeliveryDrivers,Behavior-GroceryExcuseToGetOut,Behavior-NotMindGroceriesOnline,Behavior-InPersonIsChore,ECommerce-Food-HardToKnowExactly,ECommerce-Food-WorryAboutTheft,Online-Ease
count,89.000000,89.000000,89.000000,89.000000,89.000000,89.000000,89.000000,89.000000,89.000000,89.000000,...,89.000000,89.000000,89.000000,89.000000,89.000000,89.000000,89.000000,89.000000,89.000000,89.000000
mean,0.247191,3.741573,2.404494,0.640449,0.325843,0.033708,0.550562,4.359551,3.820225,0.606742,...,4.449438,4.325843,3.292135,3.719101,3.325843,2.674157,2.741573,4.011236,2.842697,4.112360
std,0.433823,2.192591,1.212802,0.482588,0.471344,0.181499,0.500255,0.932338,1.163476,0.491241,...,0.941604,0.719458,0.842106,0.738376,1.105516,1.259286,1.201591,0.790489,1.096470,0.803942
min,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,...,1.000000,2.000000,1.000000,1.000000,1.000000,1.000000,1.000000,2.000000,1.000000,1.000000
25%,0.000000,2.000000,2.000000,0.000000,0.000000,0.000000,0.000000,4.000000,3.000000,0.000000,...,4.000000,4.000000,3.000000,3.000000,3.000000,2.000000,2.000000,3.000000,2.000000,4.000000
50%,0.000000,5.000000,2.000000,1.000000,0.000000,0.000000,1.000000,4.000000,4.000000,1.000000,...,5.000000,4.000000,3.000000,4.000000,4.000000,2.000000,2.000000,4.000000,3.000000,4.000000
75%,0.000000,5.000000,3.000000,1.000000,1.000000,0.000000,1.000000,5.000000,5.000000,1.000000,...,5.000000,5.000000,4.000000,4.000000,4.000000,4.000000,4.000000,5.000000,4.000000,5.000000
max,1.000000,6.000000,5.000000,1.000000,1.000000,1.000000,1.000000,6.000000,5.000000,1.000000,...,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000


### Frequency of grocery shopping negatively impact the adoption. Causal relationship between behaviors and attitudes.

In [161]:
X = adoptG.iloc[:,1:]
Y = adoptG.iloc[:,0]
X = sm.add_constant(X) #?
est = sm.Logit(Y, X).fit()
print(est.summary())

Optimization terminated successfully.
         Current function value: 0.270430
         Iterations 9
                           Logit Regression Results                           
Dep. Variable:                  adopt   No. Observations:                   89
Model:                          Logit   Df Residuals:                       65
Method:                           MLE   Df Model:                           23
Date:                Sun, 28 Mar 2021   Pseudo R-squ.:                  0.5164
Time:                        17:25:08   Log-Likelihood:                -24.068
converged:                       True   LL-Null:                       -49.771
Covariance Type:            nonrobust   LLR p-value:                 0.0006004
                                       coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------------------
const                              -11.7506      9.050     -1.29

/Users/apple/anaconda3/lib/python3.6/site-packages/numpy/core/fromnumeric.py:2580: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


In [162]:
adoptG = pastNeverG[pastNeverG['adopt'] != 999][['adopt',
                                                 'May-TelecommutingDays',
                                                 'NumberInHousehold',
                                                 'LiveWithSpouse',
                                                 'LiveWithChildren',
                                                 'LiveWithParents',
                                                 #'LiveWithGrandparents',
                                                 'May-Shared-Groceries',
                                                 'May_Freq_Groceries',
                                                 'May-Ease-Groceries',
                                                'InstoreTrips_Less',
                                                 'InstoreSize_Less',
                                                 'InstoreDist_Less',
                                                 'InstoreMode_Drive alone']]

In [163]:
adoptG = adoptG[~np.isin(adoptG, [-9998, 997,-1]).any(axis=1)]

In [164]:
X = adoptG.iloc[:,1:]
Y = adoptG.iloc[:,0]
X = sm.add_constant(X) #?
est = sm.Logit(Y, X).fit()
print(est.summary())

Optimization terminated successfully.
         Current function value: 0.460863
         Iterations 6
                           Logit Regression Results                           
Dep. Variable:                  adopt   No. Observations:                   91
Model:                          Logit   Df Residuals:                       78
Method:                           MLE   Df Model:                           12
Date:                Sun, 28 Mar 2021   Pseudo R-squ.:                  0.1668
Time:                        17:32:40   Log-Likelihood:                -41.939
converged:                       True   LL-Null:                       -50.332
Covariance Type:            nonrobust   LLR p-value:                    0.1578
                              coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------------
const                       2.9533      1.990      1.484      0.138      -0.947   

/Users/apple/anaconda3/lib/python3.6/site-packages/numpy/core/fromnumeric.py:2580: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


In [165]:
adoptG = pastNeverG[pastNeverG['adopt'] != 999][['adopt', 'COVID-PPEReducesContraction', 'COVID-PPEReducesTransmission', 
                  'COVID-AbleSocialDistance', 'Trust-GroceryDeliveryCompanies',
                  'Trust-GroceryDeliveryDrivers', 'Behavior-GroceryExcuseToGetOut',
                  'Behavior-NotMindGroceriesOnline', 'Behavior-InPersonIsChore', 
                  'ECommerce-Food-HardToKnowExactly', 'ECommerce-Food-WorryAboutTheft', 'Online-Ease']]

In [166]:
# Remove record with response -9998 or 997, reduce to 87 records
adoptG = adoptG[~np.isin(adoptG, [-9998, 997]).any(axis=1)]

In [167]:
adoptG.describe()

,adopt,COVID-PPEReducesContraction,COVID-PPEReducesTransmission,COVID-AbleSocialDistance,Trust-GroceryDeliveryCompanies,Trust-GroceryDeliveryDrivers,Behavior-GroceryExcuseToGetOut,Behavior-NotMindGroceriesOnline,Behavior-InPersonIsChore,ECommerce-Food-HardToKnowExactly,ECommerce-Food-WorryAboutTheft,Online-Ease
count,150.000000,150.000000,150.000000,150.000000,150.000000,150.000000,150.00000,150.000000,150.000000,150.000000,150.000000,150.000000
mean,0.340000,4.046667,4.473333,4.360000,3.366667,3.733333,3.24000,2.760000,2.713333,3.953333,2.686667,4.153333
std,0.475296,1.088911,0.848976,0.678332,0.830757,0.701548,1.19664,1.267455,1.166286,0.846126,1.118104,0.783527
min,0.000000,1.000000,1.000000,2.000000,1.000000,1.000000,1.00000,1.000000,1.000000,1.000000,1.000000,1.000000
25%,0.000000,4.000000,4.000000,4.000000,3.000000,3.000000,2.00000,2.000000,2.000000,3.250000,2.000000,4.000000
50%,0.000000,4.000000,5.000000,4.000000,3.000000,4.000000,3.50000,3.000000,2.000000,4.000000,2.000000,4.000000
75%,1.000000,5.000000,5.000000,5.000000,4.000000,4.000000,4.00000,4.000000,4.000000,5.000000,4.000000,5.000000
max,1.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.00000,5.000000,5.000000,5.000000,5.000000,5.000000


In [169]:
X = adoptG.iloc[:,1:]
Y = adoptG.iloc[:,0]
X = sm.add_constant(X) #?
est = sm.Logit(Y, X).fit()
print(est.summary())

Optimization terminated successfully.
         Current function value: 0.470243
         Iterations 6
                           Logit Regression Results                           
Dep. Variable:                  adopt   No. Observations:                  150
Model:                          Logit   Df Residuals:                      138
Method:                           MLE   Df Model:                           11
Date:                Sun, 28 Mar 2021   Pseudo R-squ.:                  0.2664
Time:                        17:35:40   Log-Likelihood:                -70.536
converged:                       True   LL-Null:                       -96.155
Covariance Type:            nonrobust   LLR p-value:                 3.745e-07
                                       coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------------------
const                               -7.0094      2.670     -2.62

/Users/apple/anaconda3/lib/python3.6/site-packages/numpy/core/fromnumeric.py:2580: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


In [141]:
keepG = oldUserG[oldUserG['keep'] != 999][['keep',
                                                 'May-TelecommutingDays',
                                                 'NumberInHousehold',
                                                 'LiveWithSpouse',
                                                 'LiveWithChildren',
                                                 'LiveWithParents',
                                                 #'LiveWithGrandparents',
                                                 #'May-Shared-Groceries',
                                                 'May_Freq_Groceries',
                                                 'May-Ease-Groceries',
                                          'COVID-PPEReducesContraction', 
                                           'COVID-PPEReducesTransmission', 
                                          'COVID-AbleSocialDistance', 
                                           'Trust-GroceryDeliveryCompanies',
                                            'Trust-GroceryDeliveryDrivers', 
                                           'Behavior-GroceryExcuseToGetOut',
                                            'Behavior-NotMindGroceriesOnline', 
                                               'Behavior-InPersonIsChore', 
                                          'ECommerce-Food-HardToKnowExactly', 
                                           'ECommerce-Food-WorryAboutTheft', 
                                          'Attitude-HaveEnoughFood', 'Online-Ease']]

In [142]:
keepG = keepG[~np.isin(keepG, [-9998, 997, -1]).any(axis=1)]

In [143]:
keepG.describe()

,keep,May-TelecommutingDays,NumberInHousehold,LiveWithSpouse,LiveWithChildren,LiveWithParents,May_Freq_Groceries,May-Ease-Groceries,COVID-PPEReducesContraction,COVID-PPEReducesTransmission,COVID-AbleSocialDistance,Trust-GroceryDeliveryCompanies,Trust-GroceryDeliveryDrivers,Behavior-GroceryExcuseToGetOut,Behavior-NotMindGroceriesOnline,Behavior-InPersonIsChore,ECommerce-Food-HardToKnowExactly,ECommerce-Food-WorryAboutTheft,Attitude-HaveEnoughFood,Online-Ease
count,47.000000,47.000000,47.000000,47.000000,47.000000,47.000000,47.000000,47.000000,47.00000,47.000000,47.000000,47.000000,47.000000,47.000000,47.000000,47.000000,47.000000,47.000000,47.000000,47.000000
mean,0.829787,3.297872,2.468085,0.659574,0.425532,0.021277,3.808511,3.829787,3.93617,4.489362,4.361702,3.553191,4.042553,3.000000,3.914894,3.361702,3.787234,2.595745,4.574468,4.446809
std,0.379883,2.244945,1.100416,0.478975,0.499769,0.145865,1.361558,1.129046,1.14975,0.748109,0.673264,0.879934,0.588197,1.302172,1.230585,1.150152,0.931016,1.076621,0.499769,0.618853
min,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,2.000000,1.00000,2.000000,2.000000,1.000000,2.000000,1.000000,1.000000,1.000000,2.000000,1.000000,4.000000,2.000000
25%,1.000000,0.500000,2.000000,0.000000,0.000000,0.000000,4.000000,3.000000,3.00000,4.000000,4.000000,3.000000,4.000000,2.000000,3.000000,2.000000,3.000000,2.000000,4.000000,4.000000
50%,1.000000,4.000000,2.000000,1.000000,0.000000,0.000000,4.000000,4.000000,4.00000,5.000000,4.000000,4.000000,4.000000,3.000000,4.000000,3.000000,4.000000,2.000000,5.000000,4.000000
75%,1.000000,5.000000,3.000000,1.000000,1.000000,0.000000,4.000000,5.000000,5.00000,5.000000,5.000000,4.000000,4.000000,4.000000,5.000000,4.000000,4.500000,3.000000,5.000000,5.000000
max,1.000000,6.000000,5.000000,1.000000,1.000000,1.000000,7.000000,5.000000,5.00000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000


### More people in the household, more likely to keep E-grocery. In store shopping frequency has a negative impact on the intention to keep E-grocery.

In [144]:
X = keepG.iloc[:,1:]
Y = keepG.iloc[:,0]
X = sm.add_constant(X) #?
est = sm.Logit(Y, X).fit()
print(est.summary())

         Current function value: 0.000000
         Iterations: 35
                           Logit Regression Results                           
Dep. Variable:                   keep   No. Observations:                   47
Model:                          Logit   Df Residuals:                       27
Method:                           MLE   Df Model:                           19
Date:                Sat, 27 Mar 2021   Pseudo R-squ.:                   1.000
Time:                        18:51:46   Log-Likelihood:            -9.9369e-06
converged:                      False   LL-Null:                       -21.443
Covariance Type:            nonrobust   LLR p-value:                  0.001344
                                       coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------------------
const                               -4.7891   2.07e+10  -2.32e-10      1.000   -4.05e+10    4.05e+10

/Users/apple/anaconda3/lib/python3.6/site-packages/numpy/core/fromnumeric.py:2580: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
/Users/apple/anaconda3/lib/python3.6/site-packages/statsmodels/base/model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)


In [145]:
keepG = oldUserG[oldUserG['keep'] != 999][['keep',
                                                 'May-TelecommutingDays',
                                                 'NumberInHousehold',
                                                 'LiveWithSpouse',
                                                 'LiveWithChildren',
                                                 'LiveWithParents',
                                                 #'LiveWithGrandparents',
                                                 #'May-Shared-Groceries',
                                                 'May_Freq_Groceries',
                                                 'May-Ease-Groceries',
                                          ]]

In [146]:
keepG = keepG[~np.isin(keepG, [-9998, 997, -1]).any(axis=1)]

In [147]:
X = keepG.iloc[:,1:]
Y = keepG.iloc[:,0]
X = sm.add_constant(X) #?
est = sm.Logit(Y, X).fit()
print(est.summary())

         Current function value: 0.310533
         Iterations: 35
                           Logit Regression Results                           
Dep. Variable:                   keep   No. Observations:                   48
Model:                          Logit   Df Residuals:                       40
Method:                           MLE   Df Model:                            7
Date:                Sat, 27 Mar 2021   Pseudo R-squ.:                  0.3108
Time:                        18:52:03   Log-Likelihood:                -14.906
converged:                      False   LL-Null:                       -21.627
Covariance Type:            nonrobust   LLR p-value:                   0.06203
                            coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                    10.4849      5.330      1.967      0.049       0.038      20.932
May-TelecommutingDays     0.2221

/Users/apple/anaconda3/lib/python3.6/site-packages/numpy/core/fromnumeric.py:2580: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
/Users/apple/anaconda3/lib/python3.6/site-packages/statsmodels/base/model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)


### 3) Eclothing

In [119]:
pastNeverC = count_('Clothing')[8]
oldUserC = count_('Clothing')[7]
nowUserC = count_('Clothing')[13]

In [121]:
pastNeverC['adopt'] = pastNeverC.apply(lambda row: adoption(row, 'Clothing'), axis = 1)
oldUserC['keep'] = oldUserC.apply(lambda row: adoption(row, 'Clothing'), axis = 1)
nowUserC['old'] = nowUserC.apply(lambda row: oldnew(row, 'Clothing'), axis = 1)

/Users/apple/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/Users/apple/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/apple/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the docum

In [122]:
adoptC = pastNeverC[pastNeverC['adopt'] != 999][['adopt',
                                                 'NonFood-OnlineRelaxing',
 'NonFood-EnjoyOnlineClothing',
 #'NonFood-EnjoyOnlineElectronice',
 'NonFood-SearchForBestDeals',
 'NonFood-EnjoyBrowsingWOBuying',
 'NonFood-InPersonClothingChore',
 'NonFood-RetailExcuseToGetOut',
 'ECommerce-NonFood-EnvironmentallyFriendly',
 'ECommerce-NonFood-SupportLocal',
 'ECommerce-NonFood-SavesTime',
 'ECommerce-NonFood-SavesMoney',
 'ECommerce-NonFood-HardToKnowExactly',
 'ECommerce-NonFood-WorryAboutTheft',
 'ECommerce-NonFood-CloseAttnToReviews',
 'ECommerce-NonFood-ComparingEasy',
 'ECommerce-NonFood-InstantIsCool',
 'Online-Ease']]

In [132]:
adoptC = adoptC[~np.isin(adoptC, [-9998, 997, -1]).any(axis=1)]

In [133]:
X = adoptC.iloc[:,1:]
Y = adoptC.iloc[:,0]
X = sm.add_constant(X) #?
est = sm.Logit(Y, X).fit()
print(est.summary())

Optimization terminated successfully.
         Current function value: 0.518699
         Iterations 6
                           Logit Regression Results                           
Dep. Variable:                  adopt   No. Observations:                   41
Model:                          Logit   Df Residuals:                       24
Method:                           MLE   Df Model:                           16
Date:                Sat, 27 Mar 2021   Pseudo R-squ.:                  0.2102
Time:                        18:49:18   Log-Likelihood:                -21.267
converged:                       True   LL-Null:                       -26.925
Covariance Type:            nonrobust   LLR p-value:                    0.7895
                                                coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------------
const                                        -

/Users/apple/anaconda3/lib/python3.6/site-packages/numpy/core/fromnumeric.py:2580: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
